<a href="https://colab.research.google.com/github/RomanGustavo/Mestrado----Projeto-Carro/blob/main/TWICE_Colab_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TWICE Dataset — Colab Starter
Este notebook prepara o ambiente, baixa o **subset** do TWICE, lê `.osi`, projeta cuboids (via utilitários do repo) e mostra como visualizar radar e rodar um detector (YOLOv7) para DEMO.


CEL 1 — instalar pacotes

o que faz: pip install de dependências usadas ao longo do notebook:

opencv-python (ler/decodificar imagens JPEG dentro dos .osi, fazer vídeo),

tqdm (barrinhas de progresso),

open3d (visualizar/grav ar nuvem de pontos .ply — útil pro LiDAR),

protobuf==4.25.3 (compatível com as mensagens OSI),

matplotlib (gráficos simples).

relação com o TWICE: os arquivos do TWICE usam OSI via protobuf; sem o protobuf correto e o OpenCV, você não abre os frames da câmera nem gera os vídeos.

In [2]:
# CEL 1 — pacotes base
!pip -q install --upgrade opencv-python tqdm open3d protobuf==4.25.3 matplotlib

# Forçar protobuf 4 só para os scripts OSI/TWICE
!pip install "protobuf==4.25.3" --force-reinstall

# Depois reinstalar o grpcio-status na versão compatível com protobuf 4
!pip install "grpcio-status==1.60.0"

  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.3 which is incompatible.


  Attempting uninstall: grpcio-status
    Found existing installation: grpcio-status 1.71.2
    Uninstalling grpcio-status-1.71.2:
      Successfully uninstalled grpcio-status-1.71.2


CEL 2 — instalar/ativar o OSI (Open Simulation Interface)

o que faz: tenta import osi3. se não existir, clona o repositório oficial do OSI, gera os módulos python a partir dos .proto (com setup.py build_py) e instala com pip.

relação com o TWICE: os .osi do TWICE são mensagens protobuf do OSI (classes como SensorData e SensorDataSeries). sem o pacote osi3, você não consegue decodificar o conteúdo dos arquivos de câmera/radar/lidar/IMU.

In [3]:
# CEL 2 — Open Simulation Interface (OSI): tenta importar; se não existir, compila a partir do repo oficial
try:
    import osi3
    print("✅ osi3 já disponível")
except Exception as e:
    print("⚠️ osi3 não encontrado. Clonando OSI e instalando a partir do código-fonte...")
    !git clone -q https://github.com/OpenSimulationInterface/open-simulation-interface.git
    %cd /content/open-simulation-interface
    !python3 setup.py -q build_py
    !pip -q install .
    %cd /content
    import osi3
    print("✅ osi3 instalado")

⚠️ osi3 não encontrado. Clonando OSI e instalando a partir do código-fonte...
/content/open-simulation-interface
Traceback (most recent call last):
  File "/content/open-simulation-interface/setup.py", line 15, in <module>
    from protoc import PROTOC_EXE
ModuleNotFoundError: No module named 'protoc'
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
/content
✅ osi3 instalado


CEL 3 — clonar os scripts do repositório TWICE

o que faz: git clone de TWICEdataset/scripts e entra na pasta.

relação com o TWICE: esse repo traz utilitários prontos que já entendem a estrutura OSI/OpenLABEL do TWICE:

cuboid_project.py: projeção de cuboids 3D → imagem (usa intrínsecas/extrínsecas).

radar_osi_plot.py: leitura/plot do radar (modos cluster e object list) + trajetória.

lidar_osi2PLY.py: converter LiDAR .osi → .ply.

tutorial.ipynb: exemplos prontos.

In [4]:
# CEL 3 — clonar scripts
!git clone -q https://github.com/TWICEdataset/scripts.git
%cd /content/scripts
!ls -la

/content/scripts
total 132
drwxr-xr-x 5 root root  4096 Aug 24 14:15 .
drwxr-xr-x 1 root root  4096 Aug 24 14:15 ..
-rw-r--r-- 1 root root 16496 Aug 24 14:15 cuboid_project.py
-rw-r--r-- 1 root root  4891 Aug 24 14:15 file_selector.py
drwxr-xr-x 8 root root  4096 Aug 24 14:15 .git
-rw-r--r-- 1 root root  3317 Aug 24 14:15 lidar_osi2PLY.py
drwxr-xr-x 2 root root  4096 Aug 24 14:15 __pycache__
-rw-r--r-- 1 root root 27662 Aug 24 14:15 radar_osi_plot.py
-rw-r--r-- 1 root root  1545 Aug 24 14:15 README.md
-rw-r--r-- 1 root root   117 Aug 24 14:15 requirements.txt
drwxr-xr-x 2 root root  4096 Aug 24 14:15 Results
-rw-r--r-- 1 root root 18228 Aug 24 14:15 tutorial.ipynb
-rw-r--r-- 1 root root 15554 Aug 24 14:15 TWICE_df.csv
-rw-r--r-- 1 root root    20 Aug 24 14:15 TWICE_path.txt
-rw-r--r-- 1 root root  2350 Aug 24 14:15 vehicle_to_image.py


CEL 4 — baixar e extrair o subset do dataset

o que faz: baixa TWICEsubset.zip (~439 MB) do servidor e extrai em /content/TWICE.

relação com o TWICE: você passa a ter localmente a árvore de cenários do dataset (ex.: Scenarios/dynamic ego/.../real/daytime/.../{Camera,Radar,LiDAR,IMU ...}), com arquivos .osi por sensor e arquivos OpenLABEL .json (anotações e calibrações).

In [5]:
# CEL 4 — baixar o subset (~439 MB) e extrair
import os, zipfile, urllib.request, pathlib

subset_url = "https://twice.eletrica.ufpr.br/TWICEsubset.zip"
zip_path = "/content/TWICEsubset.zip"
out_dir = pathlib.Path("/content/TWICE")

if not os.path.exists(zip_path):
    print("⬇️ Baixando subset...")
    urllib.request.urlretrieve(subset_url, zip_path)
else:
    print("ZIP já existe. Pulando download.")

print("📦 Extraindo...")
out_dir.mkdir(parents=True, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(out_dir)

print("✅ Pronto. Dados em:", out_dir)

⬇️ Baixando subset...


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)>

CEL 5 — configurar caminhos e importar utilitários

o que faz: define DATA_ROOT=/content/TWICE e adiciona /content/scripts no sys.path. depois importa:

rotation_matrix e cuboid_corners (para compor as arestas do cubo 3D),

Camera (modelo de câmera usado na projeção).

relação com o TWICE: prepara o ambiente para projetar as anotações 3D (OpenLABEL) sobre os frames de câmera do TWICE e para acessar outros utilitários do repo.

In [ ]:
# CEL 5 — paths e imports
import sys, pathlib
DATA_ROOT = pathlib.Path("/content/TWICE")
SCRIPTS_ROOT = pathlib.Path("/content/scripts")
sys.path.insert(0, str(SCRIPTS_ROOT))

print("DATA_ROOT =", DATA_ROOT)
print("SCRIPTS_ROOT =", SCRIPTS_ROOT)

from cuboid_project import rotation_matrix, cuboid_corners
from vehicle_to_image import Camera
print("✅ utilitários importados")

CEL 6 — ler frames de câmera de um arquivo .osi e gerar um vídeo preview

o que faz (técnico):

encontra uma pasta .../Camera/ dentro do subset.

abre o .osi da câmera em modo binário e faz o loop:

lê 4 bytes com struct.unpack("I", ...) → isso é o tamanho da próxima mensagem protobuf,

lê o próximo blob de bytes (msg_len),

instancia SensorData() (classe OSI) e chama ParseFromString(msg),

obtém sd.image_data[0].image.raw_data (bytes JPEG do frame),

decodifica para numpy com cv2.imdecode(...).

guarda ~150 frames e salva um vídeo .mp4 (15 fps) com cv2.VideoWriter.

relação com o TWICE: você reconstrói as imagens da câmera a partir do arquivo OSI do TWICE (cada mensagem é um frame), e cria um preview para inspecionar rapidamente se o caso/condição climática está ok.

In [ ]:
# CEL 6 — iterar frames do .osi e salvar um MP4 (preview)
import cv2, struct
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Preferir as classes do repositório, quando disponíveis
try:
    from cuboid_project import SensorData as OSISensorData
    SensorData = OSISensorData
    from cuboid_project import SensorDataSeries  # se necessário em outros sensores
except Exception as e:
    SensorData = None
    print("Aviso: classes de Sensor do repo não importadas:", e)

def iter_camera_frames(osi_path: Path):
    assert SensorData is not None, "SensorData do repo não disponível — verifique imports na CEL 5."
    with open(osi_path, "rb") as f:
        while True:
            len_bytes = f.read(4)
            if not len_bytes:
                break
            (msg_len,) = struct.unpack("I", len_bytes)
            msg = f.read(msg_len)
            sd = SensorData()
            sd.ParseFromString(msg)
            if sd.HasField("image_data") and len(sd.image_data) and sd.image_data[0].HasField("image"):
                img_bytes = sd.image_data[0].image.raw_data
                img_np = np.frombuffer(img_bytes, dtype=np.uint8)
                frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
                if frame is not None:
                    yield frame

# achar uma pasta /Camera no subset
scenario_dir = next(DATA_ROOT.rglob("Camera"), None)
assert scenario_dir is not None, "Não encontrei /Camera no subset. Verifique se o ZIP foi extraído corretamente."

cam_file = next(Path(scenario_dir).glob("*.osi"))
print("🎥 Câmera .osi:", cam_file)

# gerar um preview curto (≤150 frames)
frames = []
for i, fr in enumerate(iter_camera_frames(cam_file)):
    frames.append(fr)
    if i >= 150: break

assert frames, "Não foi possível decodificar frames. Verifique a instalação do OSI e o arquivo .osi."

h, w = frames[0].shape[:2]
out_path = "/content/camera_preview.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vw = cv2.VideoWriter(out_path, fourcc, 15, (w, h))
for fr in frames:
    vw.write(fr)
vw.release()
print("✅ Preview salvo em:", out_path)

CEL 7 — carregar OpenLABEL e preparar a projeção 3D→2D

o que faz:

abre o *open*label*camera*.json do cenário,

extrai a matriz intrínseca K a partir de camera_matrix_3x4 → remodela para (3×4) e usa as 3×3 primeiras colunas,

(na célula de exemplo) só escreve um texto no frame; para desenhar as arestas do cubo você usa:

os “cuboids” (posição, dimensões, orientação) em openlabel["objects"][...]["object_data"]["cuboid"]["val"],

funções do cuboid_project.py para obter os 8 cantos 3D (cuboid_corners), aplicar rotação/translação (extrínsecas) e projetar com K (intrínsecas),

desenhar arestas com cv2.line.

relação com o TWICE: o TWICE rotula em OpenLABEL (padrão da ASAM): você aproveita essas anotações para desenhar ground-truth sobre os frames, checar calibração e gerar vídeos anotados.

In [ ]:
# CEL 7 — carregar intrínsecas do OpenLABEL e desenhar uma anotação no primeiro frame
import json, cv2, numpy as np
from pathlib import Path

openlabel_file = next(Path(scenario_dir).glob("*open*label*camera*.json"), None)
print("🧾 OpenLABEL:", openlabel_file)

with open(openlabel_file, "r") as f:
    ol = json.load(f)

# Matriz intrínseca K
K = np.array(
    ol["openlabel"]["frames"][0]["frame_properties"]["Streams"]["Camera1"]["intrinsics_pinhole"]["camera_matrix_3x4"]
).reshape(3,4)[:3,:3]

# Demonstração simples (texto). Para projeção 3D->2D (arestas), use as funções do cuboid_project.py
frame0 = frames[0].copy()
cv2.putText(frame0, "Cuboids: use funcoes do cuboid_project.py para projetar 3D->2D", (30,40),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
cv2.imwrite("/content/cuboids_demo.jpg", frame0)
print("✅ Exemplo salvo em /content/cuboids_demo.jpg")

CEL 8 — visualizar radar (cluster / object list) e trajetórias

o que faz: importa radar_osi_plot.py, localiza um .osi de radar (se existir no subset daquele cenário) e orienta usar as funções do script para plotar:

cluster: nuvem de pontos (cada reflexão do radar),

object list: lista de objetos agregados pelo ECU (com velocidade/posição/RCS),

posição do ego e do alvo (IMU/GNSS) ao longo do tempo.

relação com o TWICE: permite ver como o radar “enxerga” o mesmo cenário da câmera (real e HIL), inclusive comparando modos de operação.

In [ ]:
# CEL 8 — visualizar radar (cluster/object list) — precisa de arquivo em /Radar
from importlib import reload
import os
try:
    import radar_osi_plot as rplt
    reload(rplt)
    radar_dir = (Path(scenario_dir).parent / "Radar")
    if radar_dir.exists():
        radar_osi = next(radar_dir.glob("*.osi"))
        print("📡 Radar .osi:", radar_osi)
        # Abra o arquivo radar_osi_plot.py para ver como salvar vídeo (há exemplo lá).
    else:
        print("⚠️ Este caso do subset pode não ter Radar. Teste outro cenário ou use o dataset completo.")
except Exception as e:
    print("⚠️ Import do script de radar falhou:", e)

CEL 9 — (opcional) rodar YOLOv7 como demo

o que faz: exporta ~30 frames do preview para /content/frames/*.jpg, clona o repositório do YOLOv7 e roda detect.py com yolov7.pt. salva imagens e labels preditos em /content/y7out/det.

relação com o TWICE: demonstra um baseline de detecção sobre os frames do TWICE. a partir daí, você pode comparar com GT do OpenLABEL (convertendo cuboids 3D → caixas 2D) para calcular IoU/AP, como o artigo exemplifica.

In [ ]:
# CEL 9 — (opcional) rodar YOLOv7 para DEMO
import os, cv2
os.makedirs("/content/frames", exist_ok=True)
for i, fr in enumerate(frames[:30]):
    cv2.imwrite(f"/content/frames/{i:04d}.jpg", fr)

!git clone -q https://github.com/WongKinYiu/yolov7.git
%cd /content/yolov7
!pip -q install -r requirements.txt
%cd /content/yolov7
!python detect.py --weights yolov7.pt --conf 0.25 --source /content/frames --save-txt --project /content/y7out --name det --exist-ok
print("✅ Saídas em /content/y7out/det")